
# **Tugas Menghitung Iluminasi dan Fase Bulan**

Nama : Vanda Sekar Annisa

NIM  :10320047

---

#### fraksi iluminasi Bulan tanggal 12 Agustus 2023 jam 18.00 WIB.  

In [1]:
# library
from math import *
import datetime
import pandas as pd
import numpy as np

In [1]:

y = int(input('tahun berapa   : '))
m = int(input('bulan berapa   : '))
d = int(input('tanggal berapa : '))
h = float(input('jam berapa (UT): '))

# convert to julian day
A = int(y/100)
B = 2 - A + int(A/4)

if m == 1 or m == 2:
    y = y - 1
    m = m + 12
else:
    y = y
    m = m

JD = int(365.25*(y+4716)) + int(30.6001*(m+1)) + d + B - 1524.5 + (h/24)

##print('JD pada tanggal' , datetime.date(y, m, d), 'jam', h, 'adalah', JD)

print (JD)

tahun berapa   : 2023
bulan berapa   : 2
tanggal berapa : 4
jam berapa (UT): 0
2459979.5


In [412]:
# function to find illuminated fraction
def time(JD):
    T = (JD - 2451545)/36525
    return T

T1 = time(JD)

def sunelong(T):
    D = 297.8501921 + 445267.1114034*T - 0.0018819*T**2 + T**3/545868  - T**4/113065000
    return D

D1 = sunelong(T1)

def sunano(T):
    MS = 357.5291092 + 35999.0502909*T - 0.0001536*T**2 + T**3/24490000
    return MS

MS1 = sunano(T1)

def moonano(T):
    MM = 134.9633964 + 477198.8675055*T + 0.0087414*T**2 + T**3/69699 - T**4/14712000
    return MM

MM1 = moonano(T1)

def phaseangle(MM, MS, D):
    i = 180 - D - 6.289*sin(radians(MM)) + 2.100*sin(radians(MS)) - 1.4*sin(radians(2*D - MM)) - 0.658*sin(radians(2*D)) - 0.214*sin(radians(2*MM)) - 0.110*sin(radians(D))
    return i 

i1 = phaseangle(MM1, MS1, D1)

# formula of illuminated fraction of the moon's disk
k = ((1 + cos(radians(i1)))/2)*100
print('Iluminasi Bulan pada tanggal', datetime.date(y, m, d), 'pada jam', h, 'adalah', k,'%')


Iluminasi Bulan pada tanggal 2023-08-12 pada jam 11 adalah 13.590538074834352 %


#### Tanggal terjadinya fase bulan baru, bulan setengah (kuartil awal dan kuartil akhir), dan purnama sepanjang tahun 2023.

In [4]:
df = pd.read_excel('data bulan.xlsx')

In [5]:
# find julian day
df['t'] = df.k/1236.8
df['J0'] = 2451550.09765 + 29.530588853 *df.k + (1.337*10**-4)*df.t**2 - (1.5*10**-7)*df.t**3+(7.3*10**-10)*df.t**4

In [6]:
# convert J to gregorian day
df['J'] = df.J0 + 0.5
df['F'] = df.J%1
df['z'] =  df.J - df.F
df['alpha'] = ((df.z - 1867216.25)/36524.25).apply(np.floor)
df['A2'] = (df.z + 1 + df.alpha)-((df.alpha/4).apply(np.floor))
df['B'] = df.A2 + 1524
df['C'] = ((df.B - 122.1)/365.25).apply(np.floor)
df['D'] = (365.25*df.C).apply(np.floor)
df['E'] = ((df.B - df.D)/30.6001).apply(np.floor)

# the day of the month
df['day0'] = df.B - df.D - (30.6001*df.E).apply(np.floor) + df.F
df['day'] = (df.day0).apply(np.floor)

# the month of the day
df.loc[df.E <= 14, 'month'] = df.E - 1 
df.loc[df.E == 14, 'month'] = df.E - 13
df.loc[df.E == 15, 'month'] = df.E - 13

# the year 
df.loc[df.month >= 2, 'year'] = df.C - 4716
df.loc[df.month == 1, 'year'] = df.C - 4715
df.loc[df.month == 2, 'year'] = df.C - 4715

# the time
df['hour0'] = (df.day0 - df.day)*24 + 7
df['hour'] = (df.hour0).apply(np.floor)
df['minute0'] = (df.hour0 - df.hour)*60
df['minute'] = (df.minute0).apply(np.floor)
df['second'] = ((df.minute0 - df.minute)*60).apply(np.floor)
                                                   

df.loc[df.hour >= 24, 'day'] = df.day + 1
df.loc[df.hour >= 24, 'hour'] = df.hour - 24
df.loc[df.minute >= 60, 'hour'] = df.hour + 1
df.loc[df.minute >= 60, 'minute'] = df.minute -  60

In [416]:
# print phase of the moon
print('\033[94m' + 'New Moon phase 🌑' + '\033[0m')
newmoon = df[['day', 'month', 'year', 'hour', 'minute', 'second']].head(12)
print(newmoon)

New Moon phase 🌑
     day  month    year  hour  minute  second
0   22.0    1.0  2023.0   2.0    34.0    17.0
1   20.0    2.0  2023.0  15.0    18.0    20.0
2   22.0    3.0  2023.0   4.0     2.0    23.0
3   20.0    4.0  2023.0  16.0    46.0    26.0
4   20.0    5.0  2023.0   5.0    30.0    29.0
5   18.0    6.0  2023.0  18.0    14.0    31.0
6   18.0    7.0  2023.0   6.0    58.0    34.0
7   16.0    8.0  2023.0  19.0    42.0    37.0
8   15.0    9.0  2023.0   8.0    26.0    40.0
9   14.0   10.0  2023.0  21.0    10.0    43.0
10  13.0   11.0  2023.0   9.0    54.0    46.0
11  12.0   12.0  2023.0  22.0    38.0    49.0


In [417]:
# print phase of the moon
print('\033[94m' + 'First Quarter Phase 🌓' + '\033[0m')
firstquarter = df[['day', 'month', 'year', 'hour', 'minute', 'second']].iloc[12:24]
print(firstquarter)

First Quarter Phase 🌓
     day  month    year  hour  minute  second
12  29.0    1.0  2023.0  11.0    45.0    18.0
13  28.0    2.0  2023.0   0.0    29.0    21.0
14  29.0    3.0  2023.0  13.0    13.0    23.0
15  28.0    4.0  2023.0   1.0    57.0    26.0
16  27.0    5.0  2023.0  14.0    41.0    29.0
17  26.0    6.0  2023.0   3.0    25.0    32.0
18  25.0    7.0  2023.0  16.0     9.0    35.0
19  24.0    8.0  2023.0   4.0    53.0    38.0
20  22.0    9.0  2023.0  17.0    37.0    41.0
21  22.0   10.0  2023.0   6.0    21.0    44.0
22  20.0   11.0  2023.0  19.0     5.0    47.0
23  20.0   12.0  2023.0   7.0    49.0    49.0


In [418]:
# print phase of the moon
print('\033[94m' + 'Full Moon Phase 🌕' + '\033[0m')
fullmoon = df[['day', 'month', 'year', 'hour', 'minute', 'second']].iloc[24:37]
print(fullmoon)

Full Moon Phase 🌕
     day  month    year  hour  minute  second
24   7.0    1.0  2023.0   8.0    12.0    16.0
25   5.0    2.0  2023.0  20.0    56.0    18.0
26   7.0    3.0  2023.0   9.0    40.0    21.0
27   5.0    4.0  2023.0  22.0    24.0    24.0
28   5.0    5.0  2023.0  11.0     8.0    27.0
29   3.0    6.0  2023.0  23.0    52.0    30.0
30   3.0    7.0  2023.0  12.0    36.0    33.0
31   2.0    8.0  2023.0   1.0    20.0    36.0
32  31.0    8.0  2023.0  14.0     4.0    39.0
33  30.0    9.0  2023.0   2.0    48.0    41.0
34  29.0   10.0  2023.0  15.0    32.0    44.0
35  28.0   11.0  2023.0   4.0    16.0    47.0
36  27.0   12.0  2023.0  17.0     0.0    50.0


In [419]:
# print phase of the moon
print('\033[94m' + 'Third Quarter Phase 🌗' + '\033[0m')
thirdquarter = df[['day', 'month', 'year', 'hour', 'minute', 'second']].tail(12)
print(thirdquarter)

Third Quarter Phase 🌗
     day  month    year  hour  minute  second
37  14.0    1.0  2023.0  17.0    23.0    16.0
38  13.0    2.0  2023.0   6.0     7.0    19.0
39  14.0    3.0  2023.0  18.0    51.0    22.0
40  13.0    4.0  2023.0   7.0    35.0    25.0
41  12.0    5.0  2023.0  20.0    19.0    28.0
42  11.0    6.0  2023.0   9.0     3.0    31.0
43  10.0    7.0  2023.0  21.0    47.0    34.0
44   9.0    8.0  2023.0  10.0    31.0    36.0
45   7.0    9.0  2023.0  23.0    15.0    39.0
46   7.0   10.0  2023.0  11.0    59.0    42.0
47   6.0   11.0  2023.0   0.0    43.0    45.0
48   5.0   12.0  2023.0  13.0    27.0    48.0
